In [1]:
import os

In [2]:
%pwd

'c:\\projects\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\projects\\Text-Summarizer'

In [5]:
#entity for the model evaluation

from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_path: Path
    

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directories, read_yaml


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(str(self.config.artifacts_root))])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([Path(str(config.root_dir))])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(str(config.root_dir)),
            data_path=Path(str(config.data_path)),
            model_path=Path(str(config.model_path)),
            tokenizer_path=Path(str(config.tokenizer_path)),
            metric_file_path=Path(str(config.metric_file_name)),
        )
        return model_evaluation_config

In [8]:
!pip install evaluate datasets transformers torch sentencepiece rouge-score nltk



In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm


c:\Users\ErenYager\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#update component is the stage where you use the notebook code first time
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    #evaluation
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]


    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, device=None,
                                    column_text="article",
                                    column_summary="highlights"):
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                                padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                        attention_mask=inputs["attention_mask"].to(device),
                                        length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                    clean_up_tokenization_spaces=True)
                                for s in summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        tokenizer = AutoTokenizer.from_pretrained(str(self.config.tokenizer_path))
        model = AutoModelForSeq2SeqLM.from_pretrained(str(self.config.model_path)).to(device)

        dataset_samsum_pt = load_from_disk(str(self.config.data_path))

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        metric = evaluate.load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"][0:10],
            metric,
            model,
            tokenizer,
            batch_size=2,
            column_text="dialogue",
            column_summary="summary"
        )

        rouge_dict = {rn: score[rn] for rn in rouge_names}
        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(self.config.metric_file_path, index=False)


In [11]:
#create the pipeline for model evaluation
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()

    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception :
    raise


[2026-01-09 12:54:50,208: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-09 12:54:50,216: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-09 12:54:50,219: INFO: common: created directory at: artifacts]
[2026-01-09 12:54:50,221: INFO: common: created directory at: artifacts\model_evaluation]


100%|██████████| 5/5 [1:09:05<00:00, 829.07s/it]


[2026-01-09 14:04:00,259: INFO: rouge_scorer: Using default tokenizer.]
